## to do

각 hdo 직영 주유소들 기준으로 반경 1km 내의 경쟁사 주유소들과의 게시가 차이를 주차별로 계산

    1. 특정 이름을 가진 sample들을 추출하여 모든 정보를 list로 뽑아내기. 이후 사용자가 올바른 것 선택
        input args: name
        output: [all arrays with target name]
    
    2. 1에서 뽑은 리스트에서 타겟 index를 골라주면, 주변 (distance)km 주유소들의 정보를 모두 뽑아내기
        input args: index, distance
        output: [all arrays with target stations]
        
    3. 특정 시점에 근처 경쟁 주유소들의 게시가격 평균 구하기
        input args: date
        output: (mean(G), mean(D))

    4. Mapping class 구현 (folium)
        input args: dateframe
        output: map (.html)

In [1]:
import pandas as pd
import numpy as np
from haversine import haversine
import folium
import os
from mj_tools import check_price

In [2]:
df = pd.read_csv('hdo_data_200801.csv')
print(df.shape)
df.head(3)

(50651, 11)


,Unnamed: 0,name,address,date,brand,do_brand,self,G,D,lat,lon
0,0,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 1주,SK에너지,SK에너지,셀프,1238.00,1038.00,37.755264,128.896541
1,1,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 2주,SK에너지,SK에너지,셀프,1273.00,1073.00,37.755264,128.896541
2,2,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 3주,SK에너지,SK에너지,셀프,1290.14,1090.14,37.755264,128.896541


In [3]:
# csv에서 불러왔을 때 생기는 index 제거, 한번만 실행해야 함
df = df.drop('Unnamed: 0', axis=1)
print('total data samples: {}'.format(df.shape))
df_rm = df.drop_duplicates(['address'])
print('drop duplicated: {}'.format(df_rm.shape))
df_rm.head(3)

total data samples: (50651, 10)
drop duplicated: (10112, 10)


,name,address,date,brand,do_brand,self,G,D,lat,lon
0,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 1주,SK에너지,SK에너지,셀프,1238.00,1038.00,37.755264,128.896541
5,(주)명진에너지 사천지점,강원도 강릉시 사천면 동해대로 3576,2020년 06월 1주,SK에너지,SK에너지,셀프,1275.57,1075.57,37.805112,128.861278
10,(주)삼보물류삼보주유소,강원 강릉시 동해대로 3970,2020년 06월 1주,S-OIL,S-OIL,일반,1262.71,1069.86,37.837395,128.844474


> 50651개의 샘플 중 주소 기준으로 고유한 주유소는 10112개

#### test1

In [4]:
name1 = '(주)동해에너지주유소'
name2 = '황제주유소'
name3 = '사우디주유소'
name4 = '두바이주유소'
name5 = '문성골주유소'
name6 = 'SK에너지(주) 이가주유소'

In [5]:
# class에 기본이 되는 DataFrame 입력
t = check_price(df)

In [6]:
# '황제주유소'에 관한 정보들
t.check_duplicate(name2)

[array(['황제주유소', '경기 이천시 부발읍 경충대로 2064', '2020년 06월 1주', 'SK에너지', 'SK에너지',
        '일반', 1409.0, 1241.0, 37.249522999999996, 127.49142169999999],
       dtype=object),
 array(['황제주유소', '경북 경산시 진량읍 황제1길 100', '2020년 06월 1주', 'S-OIL', 'S-OIL',
        '일반', 1318.0, 1118.0, 35.86171810000001, 128.8183953], dtype=object),
 array(['황제주유소', '충북 충주시 신니면 중원대로 5775', '2020년 06월 1주', 'GS칼텍스', 'GS칼텍스',
        '일반', 1307.14, 1107.14, 37.0021292, 127.673717], dtype=object)]

In [7]:
# 위의 세 번째(index=2) 충북 충주의 황제주유소 기준으로 반경 3km의 주유소
len(t.get_near_by_idx(2, 3))

2

In [8]:
# 해당 날짜의 근처 경쟁사 가격 정보 추출
t.get_average(2020, 6, 3)

지정한 날짜는: 2020년 06월 3주
해당 날짜의 근처 경쟁사들의 정보:
[array(['국원주유소', '충북 충주시 신니면 신덕로 339', '2020년 06월 3주', 'S-OIL', 'S-OIL',
       '일반', 1399.0, 1199.0, 37.0008019, 127.69026170000001], dtype=object)]


(1399.0, 1199.0)

#### test2

In [9]:
t.check_duplicate(name3)

[array(['사우디주유소', '경기 용인시 처인구 백암면 죽양대로 685', '2020년 06월 1주', 'SK에너지',
        'SK에너지', '일반', 1299.0, 1166.86, 37.1327982, 127.4050438],
       dtype=object),
 array(['사우디주유소', '경남 진주시 천수로 122 (주약동)', '2020년 06월 1주', 'SK에너지', 'SK에너지',
        '셀프', 1247.57, 1047.57, 35.1761837, 128.08758840000002],
       dtype=object)]

In [10]:
t.get_near_by_idx(0, 3)

[array(['비룡주유소', '경기 안성시 일죽면 죽양대로 530', '2020년 06월 1주', 'S-OIL', 'S-OIL',
        '일반', 1284.0, 1085.43, 37.123882200000004, 127.41831909999999],
       dtype=object),
 array(['삼보주유소', '경기 안성시 일죽면 죽양대로 505', '2020년 06월 1주', 'GS칼텍스', 'GS칼텍스',
        '일반', 1244.14, 1045.71, 37.122006400000004, 127.4201666],
       dtype=object),
 array(['은석주유소', '경기 안성시 일죽면 사실로 335', '2020년 06월 1주', 'GS칼텍스', 'GS칼텍스',
        '일반', 1341.86, 1141.86, 37.14131920000001, 127.4365218],
       dtype=object),
 array(['믿음가득주유소', '경기 용인시 처인구 백암면 죽양대로 820', '2020년 06월 1주', '현대오일뱅크',
        '현대오일뱅크', '일반', 1296.57, 1093.71, 37.1409844, 127.3937652],
       dtype=object),
 array(['백봉주유소', '경기 용인시 처인구 백암면 백암로 23', '2020년 06월 1주', '현대오일뱅크',
        '현대오일뱅크', '일반', 1286.43, 1089.43, 37.151439, 127.3858134],
       dtype=object)]

In [11]:
t.get_average(2020, 6, 4)

지정한 날짜는: 2020년 06월 4주
해당 날짜의 근처 경쟁사들의 정보:
[array(['비룡주유소', '경기 안성시 일죽면 죽양대로 530', '2020년 06월 4주', 'S-OIL', 'S-OIL',
       '일반', 1340.71, 1143.57, 37.123882200000004, 127.41831909999999],
      dtype=object), array(['삼보주유소', '경기 안성시 일죽면 죽양대로 505', '2020년 06월 4주', 'GS칼텍스', 'GS칼텍스',
       '일반', 1330.43, 1129.57, 37.122006400000004, 127.4201666],
      dtype=object), array(['은석주유소', '경기 안성시 일죽면 사실로 335', '2020년 06월 4주', 'GS칼텍스', 'GS칼텍스',
       '일반', 1389.0, 1189.0, 37.14131920000001, 127.4365218], dtype=object)]


(1353.38, 1154.0466666666666)

## Mapping Test

- mapping gs칼텍스 and S-oil self stations

In [12]:
from mj_tools import mapping

In [13]:
df_gs = df_rm.loc[(df_rm['brand']=='GS칼텍스') & (df_rm['self']=='셀프')]
df_soil = df_rm.loc[(df_rm['brand']=='S-OIL') & (df_rm['self']=='셀프')]

print('GS칼텍스 데이터프레임: {}'.format(df_gs.shape))
print('S-OIL 데이터프레임: {}'.format(df_soil.shape))

df_gs.head()

GS칼텍스 데이터프레임: (891, 10)
S-OIL 데이터프레임: (842, 10)


,name,address,date,brand,do_brand,self,G,D,lat,lon
30,GS교동택지주유소,강원 강릉시 교동하슬라로 233,2020년 06월 1주,GS칼텍스,GS칼텍스,셀프,1335.14,1146.14,NaN,NaN
100,기분좋은주유소,강원 강릉시 경강로 1843 (홍제동),2020년 06월 1주,GS칼텍스,GS칼텍스,셀프,1251.14,1071.14,37.746458,128.870991
110,남산주유소,강원 강릉시 남부로 136 (노암동),2020년 06월 1주,GS칼텍스,GS칼텍스,셀프,1246.43,1056.86,37.741989,128.895356
175,사천주유소,강원 강릉시 사천면 동해대로 3662,2020년 06월 1주,GS칼텍스,GS칼텍스,셀프,1277.00,1077.00,37.812005,128.856942
281,지에스칼텍스㈜ 죽헌동주유소,강원 강릉시 동해대로 3373 (죽헌동),2020년 06월 1주,GS칼텍스,GS칼텍스,셀프,1272.29,1072.29,37.789760,128.867813


In [14]:
df_gs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 30 to 50434
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      891 non-null    object 
 1   address   891 non-null    object 
 2   date      891 non-null    object 
 3   brand     891 non-null    object 
 4   do_brand  891 non-null    object 
 5   self      891 non-null    object 
 6   G         891 non-null    float64
 7   D         891 non-null    float64
 8   lat       871 non-null    float64
 9   lon       871 non-null    float64
dtypes: float64(4), object(6)
memory usage: 76.6+ KB


In [15]:
# raise ValueError if there is any NaN in table
df_gs.dropna(subset=['lat', 'lon'], inplace=True)
df_soil.dropna(subset=['lat', 'lon'], inplace=True)

In [16]:
df_gs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 100 to 50434
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      871 non-null    object 
 1   address   871 non-null    object 
 2   date      871 non-null    object 
 3   brand     871 non-null    object 
 4   do_brand  871 non-null    object 
 5   self      871 non-null    object 
 6   G         871 non-null    float64
 7   D         871 non-null    float64
 8   lat       871 non-null    float64
 9   lon       871 non-null    float64
dtypes: float64(4), object(6)
memory usage: 74.9+ KB


In [17]:
map = mapping()

In [18]:
'''
각각 추가
color options: ['beige', 'black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 
                'darkpurple', 'darkred', 'gray', 'green', 'lightblue', 'lightgray',                     'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red', 'white']
'''

map.add_circle(df=df_gs, color='darkgreen', radius=5)
map.add_circle(df=df_soil, color='orange', radius=5)

added 871 points!
added 821 points!


In [19]:
map.export_map('test')